In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [2]:
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(torch.cuda.is_available())

0
4
GeForce RTX 2080 Ti
True


In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# 全连接神经网络带有一个隐藏层
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [7]:
# 使用多GPU，奇怪的是反而速度变慢了   单GPU：time cost: 37.627745389938354  多GPU：time cost: 56.995373249053955
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  
# if torch.cuda.device_count() > 1:
 
#     model = nn.DataParallel(model, device_ids=[0,1,2])
    
#     model.to(device)

In [8]:
import time
# Train the model
sT = time.time()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device) # 将数据放到GPU上，以后的运算都在GPU上进行，加快速度 注意保证参与运算的数据都这么处理
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
eT = time.time()
print("time cost:", eT-sT)

Epoch [1/5], Step [100/600], Loss: 0.0447
Epoch [1/5], Step [200/600], Loss: 0.0330
Epoch [1/5], Step [300/600], Loss: 0.0280
Epoch [1/5], Step [400/600], Loss: 0.0294
Epoch [1/5], Step [500/600], Loss: 0.0815
Epoch [1/5], Step [600/600], Loss: 0.0546
Epoch [2/5], Step [100/600], Loss: 0.0468
Epoch [2/5], Step [200/600], Loss: 0.0242
Epoch [2/5], Step [300/600], Loss: 0.0248
Epoch [2/5], Step [400/600], Loss: 0.0453
Epoch [2/5], Step [500/600], Loss: 0.0181
Epoch [2/5], Step [600/600], Loss: 0.0051
Epoch [3/5], Step [100/600], Loss: 0.0041
Epoch [3/5], Step [200/600], Loss: 0.0064
Epoch [3/5], Step [300/600], Loss: 0.0542
Epoch [3/5], Step [400/600], Loss: 0.0163
Epoch [3/5], Step [500/600], Loss: 0.0085
Epoch [3/5], Step [600/600], Loss: 0.0118
Epoch [4/5], Step [100/600], Loss: 0.0061
Epoch [4/5], Step [200/600], Loss: 0.0126
Epoch [4/5], Step [300/600], Loss: 0.0113
Epoch [4/5], Step [400/600], Loss: 0.0061
Epoch [4/5], Step [500/600], Loss: 0.0157
Epoch [4/5], Step [600/600], Loss:

In [15]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the 10000 test images: 97.85 %
